In [1]:
import requests
import cv2

from bs4 import BeautifulSoup
import re
import os
import time
import datetime
import pandas as pd
import numpy as np

In [2]:
url = 'https://this-person-does-not-exist.com'

if os.path.isdir('./data/generated/High Res/') is False:
    os.makedirs('./data/generated/High Res/')
    
path = './data/generated/High Res/'

In [17]:
# Demonstrate retrieval of full resolution images

time_0 = time.time()

retrieve_n = 50
retrieved = 0

for image in range(retrieve_n):
    
    res = requests.get(url)
    good_status = res.status_code == 200
    wait_cycle = 0
    failed_connection = False
    
    while not good_status:
        wait_cycle += 1
        print('Site did not respond. Retrying...')
        if wait_cycle > 5:
            print(f'Could not reach site. Terminating request. \
            Loop collected {retrieved} out of {retrieve_n} image(s) prior to lost connection.')
            failed_connection = True
            break
        time.sleep(3)
        res = requests.get(url)
        good_status = res.status_code == 200

    if failed_connection:
        break

    soup = BeautifulSoup(res.text, 'lxml')

    if not os.path.exists(path):
        os.mkdir(path)
    
    image_json = soup.find('img', {'id': 'avatar'})
    image_link = url + image_json['src']
    image_name = re.findall("avatar\S+.jpg", image_link)[0]
    image_path = path + '/' + image_name
    image_res = requests.get(image_link)

    if image_res.status_code != 200:
        continue

    file = open(image_path, 'wb')
    file.write(image_res.content)
    file.close()
    
    retrieved += 1
    
    time.sleep(1)

    if (image + 1) % (retrieve_n//5) == 0:
        print(f'At {round(((image + 1) / retrieve_n) * 100, 1)}% Completion: Retrieved {retrieved} of targeted {retrieve_n} images.')
        print(f'Current Execution Time: {round((time.time() - time_0), 0)} seconds. \n')

print(f'Final Results: Retrieved {retrieved} of targeted {retrieve_n} images.')
print(f'Current Execution Time: {round((time.time() - time_0), 0)} seconds.')

At 20.0% Completion: Retrieved 10 of targeted 50 images.
Current Execution Time: 40.0 seconds. 

At 40.0% Completion: Retrieved 20 of targeted 50 images.
Current Execution Time: 74.0 seconds. 

At 60.0% Completion: Retrieved 30 of targeted 50 images.
Current Execution Time: 105.0 seconds. 

At 80.0% Completion: Retrieved 40 of targeted 50 images.
Current Execution Time: 144.0 seconds. 

At 100.0% Completion: Retrieved 50 of targeted 50 images.
Current Execution Time: 180.0 seconds. 

Final Results: Retrieved 50 of targeted 50 images.
Current Execution Time: 180.0 seconds.


In [4]:
for i in range(5):
    
    wait_cycle = 0
    failed_connection = False
    
    while 100 != 200:
        wait_cycle += 1
        print('Site did not respond. Retrying...')
        if wait_cycle > 5:
            print(f'Could not reach site. Terminating request. Loop collected {i} image(s) prior to lost connection.')
            failed_connection = True
            break
        time.sleep(1)

    if failed_connection:
        break
    print(i)

Site did not respond. Retrying...
Site did not respond. Retrying...
Site did not respond. Retrying...
Site did not respond. Retrying...
Site did not respond. Retrying...
Site did not respond. Retrying...
Could not reach site. Terminating request. Loop collected 0 image(s) prior to lost connection.


In [5]:
# Image resizing from: https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/

img1 = cv2.imread('./data/generated/High Res/avatar-008390911eb28a25b6dcbbea9cb607ac.jpg', cv2.IMREAD_UNCHANGED)
img1.shape

(1024, 1024, 3)

In [6]:
width = 252
height = 252
dim = (width, height)
resized = cv2.resize(img1, dim, interpolation = cv2.INTER_AREA)

In [10]:
cv2.imshow('Image 1', resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')

image_json = soup.find('img', {'id': 'avatar'})
image_link = url + image_json['src']
image_name = re.findall("avatar\S+.jpg", image_link)[0]
image_path = path + '/' + image_name
image_res = requests.get(image_link, stream=True).raw

# raw image handling from: https://stackoverflow.com/questions/57539233/how-to-open-an-image-from-an-url-with-opencv-using-requests-from-python
image = np.asarray(bytearray(image_res.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)

width = 256
height = 256
dim = (width, height)
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

cv2.imshow('Image 1', resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
# !!IMPORTANT!! Do not 'x' out of image popup; press any key to close. Will break jupyter!

# See the difference in downscaling results based on interpolation type

res = requests.get(url)
good_status = res.status_code == 200
wait_cycle = 0
failed_connection = False

while not good_status:
    wait_cycle += 1
    print('Site did not respond. Retrying...')
    if wait_cycle > 5:
        print(f'Could not reach site. Terminating request. \
        Loop collected {retrieved} out of {retrieve_n} image(s) prior to lost connection.')
        failed_connection = True
        break
    time.sleep(3)
    res = requests.get(url)
    good_status = res.status_code == 200

soup = BeautifulSoup(res.text, 'lxml')

image_json = soup.find('img', {'id': 'avatar'})
image_link = url + image_json['src']
image_name = re.findall("avatar\S+.jpg", image_link)[0]
image_path = path + '/' + image_name
image_res = requests.get(image_link, stream=True).raw

#     if image_res.status_code != 200:
#         continue

image = np.asarray(bytearray(image_res.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)

width = 128
height = 128
dim = (width, height)
resized = cv2.resize(image, dim, interpolation = cv2.INTER_LANCZOS4)


resized_nearest = cv2.resize(image, dim, interpolation = cv2.INTER_NEAREST) # Pretty bad
resized_linear = cv2.resize(image, dim, interpolation = cv2.INTER_LINEAR) # Decent, but a little blurry
resized_area = cv2.resize(image, dim, interpolation = cv2.INTER_AREA) # Decent, but pixelated
resized_cubic = cv2.resize(image, dim, interpolation = cv2.INTER_CUBIC) # Impressive!
resized_lanczos = cv2.resize(image, dim, interpolation = cv2.INTER_LANCZOS4) # Very impressive!

# process for displaying side-by-side images from: https://www.geeksforgeeks.org/how-to-display-multiple-images-in-one-window-using-opencv-python/
horizontal = np.concatenate((resized_nearest, resized_linear, resized_area, resized_cubic, resized_lanczos), axis=1)

cv2.imshow('Image 1', horizontal)
cv2.waitKey(0)
cv2.destroyAllWindows() # !!IMPORTANT!! Do not 'x' out of image popup; press any key to close. Will break jupyter!


In [39]:
image.size

3145728

In [32]:
time_0 = time.time()
time.sleep(2)
print(f'Execution Time: {round((time.time() - time_0), 0)} seconds')

Execution Time: 2.0 seconds


In [4]:
def generate_path(path_string):
    
    if os.path.isdir(path_string) is False:
        os.makedirs(path_string)

    return path_string

In [8]:
interp = 'INTER_LANCZOS4'

In [9]:
getattr(cv2, interp)

4

In [10]:
# Low-res (128 x 128px) image collection

def collect_low_res(path, retrieve_n=500, interp='INTER_AREA'):
    """Collect downconverted 128 x 128px resolution images from the website 'this-person-does-not-exist.com'.

    Keyword arguments:
    path -- location to place collected images
    retrieve_n -- the number of images to be harvested per call of the function (default 500)
    interp -- the openCV interpolation method to be used when resizing the image \
            Options are: 'INTER_NEAREST', 'INTER_LINEAR', 'INTER_AREA', 'INTER_CUBIC', and 'INTER_LANCZOS4'
    """
    
    time_0 = time.time()

    path = path
    retrieve_n = retrieve_n
    retrieved = 0

    for image in range(retrieve_n):

        res = requests.get(url)
        good_status = res.status_code == 200
        wait_cycle = 0
        failed_connection = False

        while not good_status:
            wait_cycle += 1
            print('Site did not respond. Retrying...')
            if wait_cycle > 5:
                print(f'Could not reach site. Terminating request. \
                Loop collected {retrieved} out of {retrieve_n} image(s) prior to lost connection.')
                failed_connection = True
                break
            time.sleep(3)
            res = requests.get(url)
            good_status = res.status_code == 200

        if failed_connection:
            break

        soup = BeautifulSoup(res.text, 'lxml')

        if not os.path.exists(path):
            os.mkdir(path)

        image_json = soup.find('img', {'id': 'avatar'})
        image_link = url + image_json['src']
        image_name = re.findall("avatar\S+.jpg", image_link)[0]
        image_path = path + '/' + image_name
        image_res = requests.get(image_link, stream=True).raw

    #     if image_res.status_code != 200:
    #         continue

        image = np.asarray(bytearray(image_res.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)

        if image.size == 0 or None:
            print('Failed to collect image')
            continue

        width = 128
        height = 128
        dim = (width, height)
        resized = cv2.resize(image, dim, interpolation = getattr(cv2, interp))

        cv2.imwrite(image_path, resized)

        time.sleep(3)
        retrieved += 1

    print(f'Retrieved {retrieved} of targeted {retrieve_n} images.')
    print(f'Execution Time: {round((time.time() - time_0), 0)} seconds')
    
    return None

In [6]:
path_low_res = generate_path('./data/generated/lanczos4')

In [11]:
collect_low_res(path_low_res, retrieve_n=650, interp='INTER_LANCZOS4')

Retrieved 650 of targeted 650 images.
Execution Time: 3829.0 seconds


In [9]:
pd.read_csv('https://www.cs.columbia.edu/CAVE/databases/pubfig/download/eval_urls.txt', sep='\t')

# PubFig Dataset v1.2 - eval_urls.txt - http://www.cs.columbia.edu/CAVE/databases/pubfig/
#             person imagenum                                           url              rect                                                                         md5sum                                       
Aaron Eckhart 1      http://farm1.static.flickr.com/119/288329997_19... 248,92,338,182   a980a9e21c90ff62e57345fad53a56c8                                                NaN                                       
              2      http://farm1.static.flickr.com/35/99344798_f2ad... 267,138,419,290  8b2bc3a7a3b4a9d5826cd31ac9254924                                                NaN                                       
              4      http://2.bp.blogspot.com/_DxSfGxvclek/SH-K403dD... 32,39,94,101     994b9bfd1464936488458d2679e05520                                                NaN                                       
              5      http://2.bp.blogspot.com/_biK-MLwOHEc/RtXJ3nA4y... 75,66,183,174    d50214036344a000cdf1e68832acf33f                                                NaN                                       
...                                                                                                                                                                      ...                                       
William Macy  96     http://farm1.static.flickr.com/62/207526936_d9d... 504,174,694,364  2df2257b04cd4f3c23ce95d67e867e2c                                                NaN                                       
              97     http://farm1.static.flickr.com/98/243909925_1ae... 275,182,429,336  af91f80483ef3aa34ca2f80ab91c09b3                                                NaN                                       
              98     http://farm1.static.flickr.com/91/245712735_4d8... 908,564,1200,857 bd1e84385beed6c5dca4f90d044ddda3                                                NaN                                       
              99     http://farm1.static.flickr.com/33/39937487_72ab... 373,259,479,365  d724cfd1decebac4d6ee79e799807f88                                                NaN                                       
              100    http://farm1.static.flickr.com/31/39937508_3fec... 180,298,442,560  d1910cdf30aa90c3436d69c6e18e35f4                                                NaN                                       

[42462 rows x 1 columns]

In [10]:
real_link = 'https://www.cs.columbia.edu/CAVE/databases/pubfig/download/eval_urls.txt'
real_resp = requests.get(real_link)
output = open('./data/real/real_images.txt', 'wb')
output.write(real_resp.content)
output.close()

In [11]:
# Approach to delete lines found here: https://pynative.com/python-delete-lines-from-file/

with open('./data/real/real_images.txt', 'r') as input_data:
    lines = input_data.readlines()

In [12]:
lines[:2]

['# PubFig Dataset v1.2 - eval_urls.txt - http://www.cs.columbia.edu/CAVE/databases/pubfig/\n',
 '#\tperson\timagenum\turl\trect\tmd5sum\n']

In [13]:
with open('./data/real/real_images.txt', 'w') as input_data:
    for number, line in enumerate(lines):
        if number == 1:
            input_data.write(line.strip('#\t'))
        else:
            if number not in [0]:
                input_data.write(line)

In [14]:
with open('./data/real/real_images.txt', 'r') as input_data:
    lines = input_data.readlines()
lines[:2]

['person\timagenum\turl\trect\tmd5sum\n',
 'Aaron Eckhart\t1\thttp://farm1.static.flickr.com/119/288329997_19ebf1d7b3_o.jpg\t248,92,338,182\ta980a9e21c90ff62e57345fad53a56c8\n']

In [29]:
df_real = pd.read_csv('./data/real/real_images.txt', sep='\t')
df_real.shape

(42461, 5)

In [16]:
real_link = df_real['url'].sample(random_state=17).values[0]
real_link

'http://fanphotosource.com/harrison-ford/photos/harrison-ford-and_elizabeth-taylor/harrison-ford-elizabeth-taylor-0000101.jpg'

In [17]:
real_link = 'http://www.firstshowing.net/img/harvey-dent-believe-350w.jpg'

In [18]:
df_real.loc[0, 'rect']

'248,92,338,182'

In [19]:
rect = '237,102,333,198'
rect_vals = [int(x) for x in rect.split(',')]
rect_vals

[237, 102, 333, 198]

In [20]:
face_center = [(rect_vals[0]-rect_vals[1])//2 + rect_vals[1],(rect_vals[2]-rect_vals[3])//2 + rect_vals[3]]
face_center

[169, 265]

In [21]:
rect_height = rect_vals[0] - rect_vals[1]
rect_width = rect_vals[2] - rect_vals[3]
rect_height, rect_width

(135, 135)

In [22]:
padded_height = rect_height * 15//10
padded_height

202

In [23]:
padded_width = rect_width * 15//10
padded_width

202

In [24]:
px_square = max(padded_height, padded_width)
px_square

202

In [31]:
image_link = real_link
image_res = requests.get(image_link, stream=True).raw

# raw image handling from: https://stackoverflow.com/questions/57539233/how-to-open-an-image-from-an-url-with-opencv-using-requests-from-python
image = np.asarray(bytearray(image_res.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)

# crop image
rect = '237,102,333,198' # this will be brought in from the dataframe
rect_vals = [int(x) for x in rect.split(',')]
face_center = [(rect_vals[0]-rect_vals[1])//2 + rect_vals[1],(rect_vals[2]-rect_vals[3])//2 + rect_vals[3]]
rect_height = rect_vals[0] - rect_vals[1]
rect_width = rect_vals[2] - rect_vals[3]
padded_height = rect_height * 15//10
padded_width = rect_width * 15//10
px_square = max(padded_height, padded_width) # produce a square that is sure to capture target face
offset = px_square//2

image = image[face_center[0]-offset:face_center[0]+offset, face_center[1]-offset:rect_vals[2]+offset]

# cv2.imshow('Image 1', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

width = 256
height = 256
dim = (width, height)
resized_nearest = cv2.resize(image, dim, interpolation = cv2.INTER_NEAREST) # Pretty bad
resized_linear = cv2.resize(image, dim, interpolation = cv2.INTER_LINEAR) # Decent, but a little blurry
resized_area = cv2.resize(image, dim, interpolation = cv2.INTER_AREA) # Decent, but pixelated
resized_cubic = cv2.resize(image, dim, interpolation = cv2.INTER_CUBIC) # Impressive!
resized_lanczos = cv2.resize(image, dim, interpolation = cv2.INTER_LANCZOS4) # Very impressive!

# process for displaying side-by-side images from: https://www.geeksforgeeks.org/how-to-display-multiple-images-in-one-window-using-opencv-python/
horizontal = np.concatenate((resized_nearest, resized_linear, resized_area, resized_cubic, resized_lanczos), axis=1)

# cv2.imshow('Image 1', horizontal)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

cv2.imshow('Window 1', resized_lanczos)
cv2.imshow('Window 2', resized_nearest)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
df_real['url'].values[:5]

array(['http://farm1.static.flickr.com/119/288329997_19ebf1d7b3_o.jpg',
       'http://farm1.static.flickr.com/35/99344798_f2ad604eda_o.jpg',
       'http://2.bp.blogspot.com/_DxSfGxvclek/SH-K403dDuI/AAAAAAAAAC8/6TGJ67y8kZk/s320/Aaron%2BEckhart.jpg',
       'http://2.bp.blogspot.com/_biK-MLwOHEc/RtXJ3nA4y5I/AAAAAAAAALM/fMGxP5sRK10/s320/aaroneckhart.jpg',
       'http://3.bp.blogspot.com/_bto58WjLomw/Rk1yrB-rEXI/AAAAAAAABGo/dy33tJJsjSE/s400/aaron_eckhart3.jpg'],
      dtype=object)

## Experiment: Test whether preprocessing/interpolation of the synthetic images is 'giving away' the synthetics in the model

The 128 x 128px data set from Kaggle, while being very convenient and having my deep appreciation, does not specify how the full resolution images from the FFHQ database were downconverted. This poses a problem due to the fact that I apply a downconversion on the raw 1024 x 1024px synthetic images upon retrieval, and in turn must select an interpolation method to execute the downconversion. There is a potentiality that the methods used for downconversion between real and synthetic images differ, and that there may be artifacts to either process that are present in the images purely due to preprocessing, and could be a factor in the neural net's ability to distinguish one from the other. Here, I bring in a small set of full resolution real images and apply identical interpolation methods for the downconversion that I have applied to the raw synthetic images, so that a comparison model can be built and relative performance can be evaluated.  

#### Downconvert full resolution real images

In [48]:
def downscale_imgs(full_res_dir, destination_dir, interp, width=128, height=128):
    
    destination_dir = destination_dir + interp + '/'
    
    if not os.path.exists(destination_dir):
        os.mkdir(destination_dir)

    for image_name in os.listdir(full_res_dir):

            image_path = full_res_dir + image_name
            image = cv2.imread(image_path)
                        
            dim = (width, height)
            resized = cv2.resize(image, dim, interpolation = getattr(cv2, interp))
#             print((destination_dir + interp + '_' + image_name))
            cv2.imwrite((destination_dir + interp + '_' + image_name), resized)

In [50]:
full_res_dir = './data/real/FFHQ-high-res/69000/'
destination_dir = './data/real/FFHQ-high-res/'

In [51]:
downscale_imgs(full_res_dir, destination_dir, 'INTER_AREA')

In [52]:
downscale_imgs(full_res_dir, destination_dir, 'INTER_LANCZOS4')